# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD4 model for fitting the actual Chilean data in order to make projections in bed usage, amount of infected and deaths. 
This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [2]:
initdate = datetime(2020,5,15)
initdate2 = datetime(2020,8,15)
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. Meanwhile we will adjust this data for regions, then I'll add Heath sector as the minimum territorial unit for SEIRHVD models.

In [3]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Done


## Simulation Parameters

In [4]:
# Simulation parameters
tsimtot = 1000

tsim1 = (initdate2 - initdate).days 
tsim2 = tsimtot-tsim1

beta = 0.2
mu = 0.8
k = 0.1

SeroPrevFactor1=0.05
SeroPrevFactor2=0.05

expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected


## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [5]:
# Quarantines 
max_mob = 0.55
rem_mob = 0.45
t_end_date=datetime(2020,8,25)
t_end=(t_end_date-initdate).days

alpha = Quarantine(rem_mob,max_mob,qp=0,iqt=0,fqt=t_end).alpha
alpha = Quarantine(rem_mob).alpha

## Underreport
Fraction of Infected detected/reported  
    - Imi_det: fraction of Mild detected
    - Imi_as: fraction of asymptomatic detected
    
If both are 1, means that all infected are detected so I_det = I

In [6]:
Imi_det = 0.56
Ias_det = 0

## Simulation
Initialize Simulation Object, and run the simulation. 

In [7]:
simulation1 = SEIRHVD(tsim1,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor1,expinfection=expinfection,RealIC = RM,Imi_det = Imi_det,Ias_det = Ias_det)

InitialCondition Object Data


## Run simulation

In [8]:
simulation1.integr(0,tsim1,0.1)

Import odeint


SolverReturn(flag=<StatusEnum.SUCCESS: 0>, values=SolverVariables(t=array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9,
       11. , 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. ,
       12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1,
       13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 1

## Simulation 2

In [10]:
SeroPrevFactor2 = 0

In [11]:
simulation2 = SEIRHVD(tsim2,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor2,Imi_det = Imi_det,Ias_det = Ias_det,SimIC=simulation1,initdate=initdate2)

### Run simulation 2

In [12]:
simulation2.integr(0,tsim2,0.1)

Import odeint


SolverReturn(flag=<StatusEnum.SUCCESS: 0>, values=SolverVariables(t=array([0.000e+00, 1.000e-01, 2.000e-01, ..., 9.078e+02, 9.079e+02,
       9.080e+02]), y=array([[5.98928498e+03, 1.50650721e+03, 1.36493230e+01, ...,
        4.12828002e+03, 4.93120370e+02, 3.76608440e+03],
       [5.96485429e+03, 1.49987323e+03, 1.35337451e+01, ...,
        4.12828002e+03, 4.93120370e+02, 3.76770037e+03],
       [5.94052690e+03, 1.49327120e+03, 1.34191469e+01, ...,
        4.12828002e+03, 4.93120370e+02, 3.76931010e+03],
       ...,
       [2.55172035e-12, 6.53308542e-13, 9.64486896e-15, ...,
        4.12828002e+03, 4.93120370e+02, 4.16677086e+03],
       [2.53318710e-12, 6.59746656e-13, 1.88717447e-14, ...,
        4.12828002e+03, 4.93120370e+02, 4.16677086e+03],
       [2.51465386e-12, 6.66184771e-13, 2.80986205e-14, ...,
        4.12828002e+03, 4.93120370e+02, 4.16677086e+03]])), errors=SolverVariables(t=None, y=None), roots=SolverVariables(t=None, y=None), tstop=SolverVariables(t=None, y=None), me

# Analysis

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.I_d_det,label='sim2')
plt.plot(simulation1.t,simulation1.I_d_det,label='sim1')
plt.legend(loc=0)
plt.title('Daily New Infected')
plt.show()

In [14]:
plt.plot(simulation2.t+tsim1,simulation2.D,label='sim2')
plt.plot(simulation1.t,simulation1.D,label='sim1')
plt.legend(loc=0)
plt.title('Deaths')
plt.show()

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.V_cap,label='VMI capacity sim2')
plt.plot(simulation1.t,simulation1.V_cap,label='VMI capacity sim1')
plt.plot(simulation1.t,simulation1.V,label='VMI sim1')
plt.plot(simulation2.t+tsim1,simulation2.V,label='VMI sim2')
plt.legend(loc=0)
plt.title('VMI')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Vr_tot,label='Real Data')
plt.show()

In [15]:
plt.plot(simulation2.t+tsim1,simulation2.H_cap,label='sim2')
plt.plot(simulation1.t,simulation1.H_cap,label='sim1')
plt.plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UCI/UTI Beds')
plt.plot(simulation2.t+tsim1,simulation2.Hse+simulation2.Hout,label='UCI/UTI Beds')
plt.legend(loc=0)
plt.title('Hospitalization')

plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.show()

In [13]:
plt.plot(simulation2.t+tsim1,simulation2.D,label='sim2')
plt.plot(simulation1.t,simulation1.D,label='sim1')


plt.plot(simulation2.t+tsim1,simulation2.V_D_d,label='V_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Hse_D_d,label='Hse_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Icr_D_d,label='Icr_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Ise_D_d,label='Ise_D sim2')

plt.plot(simulation1.t,simulation1.V_D_d,label='V_D sim1')
plt.plot(simulation1.t,simulation1.Hse_D_d,label='Hse_D sim1')
plt.plot(simulation1.t,simulation1.Icr_D_d,label='Icr_D sim1')
plt.plot(simulation1.t,simulation1.Ise_D_d,label='Ise_D sim1')

plt.legend(loc=0)
plt.title('Deaths by cause')
plt.show()

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.H_sat,label='HSat sim2')
plt.plot(simulation1.t,simulation1.H_sat,label='HSat sim1')
plt.plot(simulation2.t+tsim1,simulation2.V_sat,label='VSat sim2')
plt.plot(simulation1.t,simulation1.V_sat,label='Vsat sim1')
plt.legend(loc=0)
plt.title('Saturation')
plt.show()

In [ ]:
simulation2.T_delta

In [ ]:
simulation1.Htot(92)

In [ ]:
simulation2.Htot(1)

In [ ]:
simulation1.

In [ ]:
self = simulation1

In [ ]:
self.pE_Ias + self.pE_Imi + self.pE_Ise + self.pE_Icr 

# Analysis - Detected infected

The following plots will show the simulation results with the real data

In [ ]:
# Days to plot (Today + ..)
days = currentday+130
index1 = np.searchsorted(simulation1.t,days)


### Errors

In [ ]:
simulation = simulation1
t_end_idx = np.where(np.array(RM.tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t_end_idx])
E_I = np.sum(abs(RM.Ir[:t_end_idx]-simulation.I_det[idx]))/(np.mean(RM.Ir[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_d_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_d_r_tr[:t_end_idx])
E_Id = np.sum(abs(RM.I_d_r[:t_end_idx]-simulation.I_d_det[idx]))/(np.mean(RM.I_d_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_ac_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_ac_r_tr[:t_end_idx])
E_Iac = np.sum(abs(RM.I_ac_r[:t_end_idx]-simulation.I_ac_det[idx]))/(np.mean(RM.I_ac_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.Br_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.Br_tr[:t_end_idx])
E_D = np.sum(abs(RM.Br[:t_end_idx]-simulation.B[idx]))/(np.mean(RM.Br[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_UCI = np.sum(abs(RM.Hr[:t_end_idx]-(simulation.Hse[idx]+simulation.Hout[idx])))/(np.mean(RM.Hr[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_VMI = np.sum(abs(RM.Vr[:t_end_idx]-simulation.V[idx]))/(np.mean(RM.Vr[:t_end_idx])*t_end_idx)

Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}


In [ ]:
Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}
print(Err)

### Plot Grid

In [ ]:


fig, axs = plt.subplots(3, 2)
axs[0,0].set_xlim(0,days)
axs[0,0].scatter(RM.tr,RM.Ir,label='Real Active Data')
axs[0,0].plot(simulation1.t[:index1],simulation1.I_det[:index1],label='Infected')
axs[0,0].plot(simulation2.t[:index2],simulation2.I_det[:index2],label='Infected')
axs[0,0].plot(simulation3.t[:index3],simulation3.I_det[:index3],label='Infected')
axs[0,0].set_title('Detectable Active Infected')
axs[0,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[0, 1].set_xlim(0,days)
axs[0, 1].plot(simulation1.t[:index1],simulation1.I_d_det[:index1],label='Daily new Infected')
axs[0, 1].plot(simulation2.t[:index2],simulation2.I_d_det[:index2],label='Daily new Infected')
axs[0, 1].plot(simulation3.t[:index3],simulation3.I_d_det[:index3],label='Daily new Infected')
axs[0, 1].scatter(RM.I_d_r_tr,RM.I_d_r,label='Daily Real Data')
#axs[0, 1].legend(loc=0)
axs[0, 1].set_title('Detected Daily infected')
axs[0,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,0].set_xlim(0,days)
axs[1,0].plot(simulation1.t[:index1],simulation1.I_ac_det[:index1],label='Detected Acummulated Infected')
axs[1,0].plot(simulation2.t[:index2],simulation2.I_ac_det[:index2],label='Detected Acummulated Infected')
axs[1,0].plot(simulation3.t[:index3],simulation3.I_ac_det[:index3],label='Detected Acummulated Infected')
axs[1,0].scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Accumulated Real Data')
axs[1,0].set_title('Detected Accumulated Infected')
axs[1,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[1,1].set_xlim(0,days)
axs[1,1].plot(simulation1.t[:index1],simulation1.B[:index1],label='Total Acummulated Deaths')
axs[1,1].plot(simulation2.t[:index2],simulation2.B[:index2],label='Total Acummulated Deaths')
axs[1,1].plot(simulation3.t[:index3],simulation3.B[:index3],label='Total Acummulated Deaths')
axs[1,1].scatter(RM.Br_tr,RM.Br,label='Real Data')
axs[1,1].set_title('Acmulated Deaths')
axs[1,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[2,0].set_xlim(0,days)
axs[2,0].plot(simulation1.t[:index1],simulation1.Hse[:index1]+simulation1.Hout[:index1],label='UCI/UTI Beds')
axs[2,0].plot(simulation2.t[:index2],simulation2.Hse[:index2]+simulation2.Hout[:index2],label='UCI/UTI Beds')
axs[2,0].plot(simulation3.t[:index3],simulation3.Hse[:index3]+simulation3.Hout[:index3],label='UCI/UTI Beds')
axs[2,0].scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
axs[2,0].scatter(RM.sochimi_tr,RM.Hr_tot,label='Capacity Data', color = 'red')
axs[2,0].set_title('UCI/UTI Usage')
axs[2,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[2,1].set_xlim(0,days)
axs[2,1].plot(simulation1.t[:index1],simulation1.V[:index1],label='VMI Usage')
axs[2,1].plot(simulation2.t[:index2],simulation2.V[:index2],label='VMI Usage')
axs[2,1].plot(simulation3.t[:index3],simulation3.V[:index3],label='VMI Usage')
axs[2,1].scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
axs[2,1].scatter(RM.sochimi_tr,RM.Vr_tot,label='Capacity Data',color = 'red')
axs[2,1].set_title('UCI/UTI Usage')
axs[2,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


In [ ]:
print([Err])

## Detected Active Infected
Here we define a Detected infected population as follows:  

 Idet = Imi*det + Ise + Icr  

where det is the proportion of mild detections


**Aun me falta corregir el valor inicial, pues este lo considera para todos los infectados**

In [ ]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_det[:index],label='Infected')
plt.legend(loc=0)
plt.title('Detectable Active Infected')
plt.show()


## Detected Daily Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d_det[:index],label='Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.title('Detected Daily infected')
plt.show()

## Detected Accumulated Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac_det[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

# Analysis - Total infected

The following plots will show the simulation results with the real data

## Total Active Infected

In [ ]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I[:index],label='Total Active Infected')
plt.legend(loc=0)
plt.title('Total Active Infected')
plt.show()


## Daily Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d[:index],label='Total Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Acummulated Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Deaths

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.B[:index],label='Total Acummulated Deaths')
plt.scatter(RM.Br_tr,RM.Br,label='Real Data')
plt.legend(loc=0)
plt.show()

## UCI/UTI

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.Hse[:index]+simulation.Hout[:index],label='UCI/UTI Beds')
plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.legend(loc=0)
plt.show()

## VMI

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.V[:index],label='VMI Usage')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.legend(loc=0)
plt.show()

# Objective Functions



In [ ]:
# Date until which we calculate the error (normally when the quarantine dynamic change)
t1_date = datetime(2020,8,15)


t1_day = (t1_date-initdate).days
t1_idx = np.where(np.array(RM.tr)>=t1_day)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t1_idx])

## Error:

In [ ]:
Err = np.sum(abs(RM.Ir[:t1_idx]-simulation.I[idx]))/np.mean(RM.Ir[:t1_idx])
print(Err)


In [ ]:
simulation.peak_date

In [ ]:
RM.initdate

# Second Part Simulation

In [ ]:
Htot=30,Vtot=20,H0=0,V0=0,B0=0,D0=0,R0=0,I0=100,I_d0=10,I_ac0=100,SeroPrevFactor=1,expinfection=0,population=1000000,InitialConditions=None, initdate = None,Imi_det = 1,Ias_det = 1,Einit=False,E0=None,E_d0=None,E_ac0=None

In [ ]:
simulation_part2 = SEIRHVD(tsim,beta,mu,alpha1,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM,Imi_det = Imi_det,Ias_det = Ias_det)

